In [8]:
import torch
import torch.nn as nn
import math
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
tf = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=tf)
trainloader = DataLoader(trainset, batch_size=256, shuffle=False, drop_last = True)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=tf)
testloader = DataLoader(testset, batch_size=256, shuffle=False, drop_last = True)

Files already downloaded and verified
Files already downloaded and verified


In [11]:
class VGG(nn.Module):
    def __init__(self, out_dim = 10):
        super(VGG, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
        )

        self.classifier = nn.Sequential(
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Linear(512, out_dim),
            nn.Softmax(dim=1)
        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                m.bias.data.zero_()

    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 512)
        x = self.classifier(x)
        return x

In [12]:
def cal_acc(model, dataloader, device):
    model.eval()
    correct, total = 0., 0.
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [13]:
def train(criterion, optimizer, model, num_epochs, trainloader, testloader, device):
    for epoch in range(num_epochs):
        model.train()
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        test_acc = cal_acc(model, testloader, device)

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Test Acc: {test_acc:.4f}")

In [14]:
model = VGG()
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 0.0005)

num_epochs = 30
train(criterion, optimizer, model, num_epochs, trainloader, testloader, device)

Epoch [1/30], Loss: 1.8342, Test Acc: 0.5578
Epoch [2/30], Loss: 1.7272, Test Acc: 0.6522
Epoch [3/30], Loss: 1.7227, Test Acc: 0.6352
Epoch [4/30], Loss: 1.6522, Test Acc: 0.6603
Epoch [5/30], Loss: 1.6534, Test Acc: 0.7292
Epoch [6/30], Loss: 1.6263, Test Acc: 0.6161
Epoch [7/30], Loss: 1.6252, Test Acc: 0.6930
Epoch [8/30], Loss: 1.6209, Test Acc: 0.7318
Epoch [9/30], Loss: 1.6047, Test Acc: 0.7330
Epoch [10/30], Loss: 1.5848, Test Acc: 0.7896
Epoch [11/30], Loss: 1.6188, Test Acc: 0.7692
Epoch [12/30], Loss: 1.5542, Test Acc: 0.7465
Epoch [13/30], Loss: 1.5582, Test Acc: 0.7658
Epoch [14/30], Loss: 1.5534, Test Acc: 0.7844
Epoch [15/30], Loss: 1.5513, Test Acc: 0.7747
Epoch [16/30], Loss: 1.5346, Test Acc: 0.7830
Epoch [17/30], Loss: 1.5475, Test Acc: 0.7940
Epoch [18/30], Loss: 1.5355, Test Acc: 0.8003
Epoch [19/30], Loss: 1.5286, Test Acc: 0.7990
Epoch [20/30], Loss: 1.5309, Test Acc: 0.7885
Epoch [21/30], Loss: 1.5251, Test Acc: 0.8039
Epoch [22/30], Loss: 1.5204, Test Acc: 0.80